In [686]:
import pandas as pd
import numpy as np

Ideja zadatka je analizirati razlike u školstvu u zemljama svijeta.
Podatke uzimam sa stranice http://data.un.org/

In [687]:
data_popc = pd.read_csv('Population Growth Rates in Urban areas and Capital cities.csv', encoding='utf-8')
data_pop = pd.read_csv('Population Growth, Fertility and Mortality.csv', encoding='utf-8')
#data_gen = pd.read_csv('Population, Surface Area and Density.csv', encoding='utf-8')
data_edu = pd.read_csv('Public Expenditure on Education.csv', encoding='utf-8')
data_gdp = pd.read_csv('GDP and GDP Per Capita.csv', encoding='utf-8')

Provjeravam za koje godine imam sve podatke. Ovdje gledam kriterij postotak BDP-a koji se troši na školstvo jer je to jedan od najvažnijih pokazatelja.

In [688]:
data_edu[data_edu['Series'] == 'Public expenditure on education (% of GDP)'].groupby(['Year']).size()

Year
2000     17
2001     13
2002     22
2003     16
2004     32
2005    113
2006     12
2007      9
2008     26
2009     27
2010    126
2011     14
2012     13
2013     23
2014     22
2015     60
2016     38
2017     55
2018      1
dtype: int64

Vidim da niti za jednu godinu nemam kompletne podatke. Zato ću raditi na podacima koji su prosjek zadnjih 10 godina, odnosno nakon 2010. godine. Iako će vrijednosti određenih pokazatelja fluktuirati tijekom ovih godina, važniji je odnos između država nego apsoutne vrijednosti.

In [689]:
for podaci in (data_popc, data_pop, data_edu, data_gdp):
    podaci.drop(podaci[podaci.Year < 2010].index, inplace=True)

Ostavljam podatke koji će mi trebati. Ostale brišem. Pogledajmo koje podatke imam.

In [690]:
print(data_popc['Series'].drop_duplicates().to_numpy())
print(data_pop['Series'].drop_duplicates().to_numpy())
print(data_edu['Series'].drop_duplicates().to_numpy())
print(data_gdp['Series'].drop_duplicates().to_numpy())

['Urban population (percent)'
 'Urban population (percent growth rate per annum)'
 'Rural population (percent growth rate per annum)'
 'Capital city population (thousands)'
 'Capital city population (as a percentage of total population)'
 'Capital city population (as a percentage of total urban population)']
['Population annual rate of increase (percent)'
 'Total fertility rate (children per women)'
 'Infant mortality for both sexes (per 1,000 live births)'
 'Maternal mortality ratio (deaths per 100,000 population)'
 'Life expectancy at birth for both sexes (years)'
 'Life expectancy at birth for males (years)'
 'Life expectancy at birth for females (years)']
['Current expenditure other than staff compensation as % of total expenditure in public institutions (%)'
 'All staff compensation as % of total expenditure in public institutions (%)'
 'Capital expenditure as % of total expenditure in public institutions (%)'
 'Expenditure by level of education: primary (as % of government expend

Dodaj koje podatke uzimam.????????

In [691]:
data_popc.drop(data_popc[data_popc.Series != 'Urban population (percent)'].index, inplace=True)
data_pop.drop(data_pop[data_pop.Series != 'Life expectancy at birth for both sexes (years)'].index, inplace = True)

In [692]:
izbaci = data_edu.Series.isin(['Expenditure by level of education: primary (as % of government expenditure)',
                               'Expenditure by level of education: secondary (as % of government expenditure)',
                               'Expenditure by level of education: tertiary (as % of government expenditure)',
                               'Expenditure by level of education: pre-primary (as % of government expenditure)'
                              ])
data_edu.drop(data_edu[izbaci].index, inplace=True)

In [693]:
data_gdp.drop(data_gdp[data_gdp.Series != 'GDP per capita (US dollars)'].index, inplace=True)

Pretvaram stupce s brojevima iz object u type float64.

In [694]:
data_popc['Value'] = data_popc['Value'].astype(str).astype(float)

In [695]:
data_gdp['Value'] = data_gdp['Value'].astype(str).str.replace(',', '.').astype(float)

In [696]:
data_gdp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 848 entries, 10 to 5788
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Region/Country/Area  848 non-null    int64  
 1   Unnamed: 1           848 non-null    object 
 2   Year                 848 non-null    int64  
 3   Series               848 non-null    object 
 4   Value                848 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 39.8+ KB


Sada grupiram vrijednosti po državama (to je stupac 'Unnamed: 1') i računam prosjek

In [697]:
a = data_popc.groupby('Unnamed: 1')['Value'].mean()

In [698]:
b = data_pop.groupby('Unnamed: 1')['Value'].mean()

In [699]:
c = data_edu.groupby(['Unnamed: 1', 'Series']).mean()

In [700]:
d = data_gdp.groupby('Unnamed: 1')['Value'].mean()

In [701]:
a.head(20)

Unnamed: 1
Afghanistan             24.666667
Albania                 56.633333
Algeria                 70.300000
American Samoa          87.333333
Andorra                 88.400000
Angola                  62.900000
Anguilla               100.000000
Antigua and Barbuda     25.266667
Argentina               91.400000
Armenia                 63.200000
Aruba                   43.200000
Australia               85.633333
Austria                 57.800000
Azerbaijan              54.600000
Bahamas                 82.700000
Bahrain                 88.966667
Bangladesh              33.800000
Barbados                31.400000
Belarus                 76.833333
Belgium                 97.866667
Name: Value, dtype: float64

In [702]:
b.head(20)

Unnamed: 1
Afghanistan            60.9765
Albania                76.5630
Algeria                74.8410
Angola                 55.2130
Antigua and Barbuda    75.8005
Argentina              75.2715
Armenia                73.3635
Aruba                  75.0190
Australia              81.9195
Austria                80.5580
Azerbaijan             70.8315
Bahamas                72.2370
Bahrain                76.0495
Bangladesh             69.8130
Barbados               78.3575
Belarus                70.8525
Belgium                80.0470
Belize                 72.0460
Benin                  59.2455
Bermuda                79.2900
Name: Value, dtype: float64

In [703]:
c.reset_index(inplace=True)

In [704]:
c.Series.drop_duplicates().to_numpy()

array(['All staff compensation as % of total expenditure in public institutions (%)',
       'Capital expenditure as % of total expenditure in public institutions (%)',
       'Current expenditure other than staff compensation as % of total expenditure in public institutions (%)',
       'Public expenditure on education (% of GDP)',
       'Public expenditure on education (% of government expenditure)'],
      dtype=object)

In [705]:
c.set_index('Unnamed: 1', inplace=True)

In [706]:
c

,Series,Region/Country/Area,Year,Value
Unnamed: 1,,,,
Afghanistan,All staff compensation as % of total expenditu...,4.0,2012.5,71.912700
Afghanistan,Capital expenditure as % of total expenditure ...,4.0,2012.5,18.955650
Afghanistan,Current expenditure other than staff compensat...,4.0,2012.5,9.131700
Afghanistan,Public expenditure on education (% of GDP),4.0,2014.0,3.543600
Afghanistan,Public expenditure on education (% of governme...,4.0,2014.0,15.079333
...,...,...,...,...
Viet Nam,Current expenditure other than staff compensat...,704.0,2011.5,31.791250
Viet Nam,Public expenditure on education (% of GDP),704.0,2011.5,5.394600
Viet Nam,Public expenditure on education (% of governme...,704.0,2011.5,17.815000


In [707]:
c[c.Series=='All staff compensation as % of total expenditure in public institutions (%)']['Value']

Unnamed: 1
Afghanistan                 71.91270
Albania                     76.09600
Andorra                     36.09150
Argentina                   84.14395
Armenia                     82.46800
                              ...   
United Kingdom              66.45955
United States of America    69.54495
Uruguay                     76.87285
Vanuatu                     77.99230
Viet Nam                    44.61140
Name: Value, Length: 135, dtype: float64

In [708]:
data = pd.DataFrame({'Urban population (percent)' : a,
              'Life expectancy at birth for both sexes (years)': b,
              'GDP per capita (US dollars)' : d,
              'All staff compensation as % of total expenditure in public institutions (%)' : c[c.Series=='All staff compensation as % of total expenditure in public institutions (%)']['Value'],
              'Capital expenditure as % of total expenditure in public institutions (%)' : c[c.Series=='Capital expenditure as % of total expenditure in public institutions (%)']['Value'],
              'Current expenditure other than staff compensation as % of total expenditure in public institutions (%)' : c[c.Series=='Current expenditure other than staff compensation as % of total expenditure in public institutions (%)']['Value'],
              'Public expenditure on education (% of GDP)' : c[c.Series=='Public expenditure on education (% of GDP)']['Value'],
              'Public expenditure on education (% of government expenditure)' : c[c.Series=='Public expenditure on education (% of government expenditure)']['Value']
             })

In [709]:
data

,Urban population (percent),Life expectancy at birth for both sexes (years),GDP per capita (US dollars),All staff compensation as % of total expenditure in public institutions (%),Capital expenditure as % of total expenditure in public institutions (%),Current expenditure other than staff compensation as % of total expenditure in public institutions (%),Public expenditure on education (% of GDP),Public expenditure on education (% of government expenditure)
Afghanistan,24.666667,60.9765,593.00000,71.9127,18.95565,9.1317,3.5436,15.079333
Albania,56.633333,76.5630,4.11550,76.0960,8.20850,15.6955,3.6963,12.457350
Algeria,70.300000,74.8410,4.15600,NaN,NaN,NaN,NaN,NaN
American Samoa,87.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,88.400000,NaN,38.03925,36.0915,1.20650,47.3752,3.1265,NaN
...,...,...,...,...,...,...,...,...
Western Sahara,86.500000,67.3060,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,34.400000,65.3505,720.57725,NaN,NaN,NaN,NaN,NaN
Zambia,41.600000,55.5205,1.38950,NaN,NaN,NaN,NaN,NaN
Zanzibar,NaN,NaN,781.25000,NaN,NaN,NaN,NaN,NaN


In [716]:
data.dropna()

,Urban population (percent),Life expectancy at birth for both sexes (years),GDP per capita (US dollars),All staff compensation as % of total expenditure in public institutions (%),Capital expenditure as % of total expenditure in public institutions (%),Current expenditure other than staff compensation as % of total expenditure in public institutions (%),Public expenditure on education (% of GDP),Public expenditure on education (% of government expenditure)
Afghanistan,24.666667,60.9765,593.00000,71.91270,18.95565,9.13170,3.54360,15.079333
Albania,56.633333,76.5630,4.11550,76.09600,8.20850,15.69550,3.69630,12.457350
Argentina,91.400000,75.2715,13.06325,84.14395,7.51305,8.34300,5.29595,14.212250
Armenia,63.200000,73.3635,3.64825,82.46800,9.29680,8.23520,3.00355,11.298850
Australia,85.633333,81.9195,55.67850,65.22720,7.66170,27.11110,5.44050,14.201700
...,...,...,...,...,...,...,...,...
United Kingdom,82.433333,80.2765,40.80375,66.45955,6.94390,26.59660,5.65615,13.471500
United States of America,81.600000,78.5665,55.91025,69.54495,9.39980,21.05525,5.20475,13.372750
Uruguay,94.900000,76.7125,14.97025,76.87285,4.15300,18.97415,4.35530,14.926100
Vanuatu,24.933333,69.1155,2.98500,77.99230,1.19360,20.81410,4.69540,11.782700


In [717]:
data2 = pd.DataFrame({'Urban population (percent)' : a,
              'Life expectancy at birth for both sexes (years)': b,
              'GDP per capita (US dollars)' : d,                          
              'Public expenditure on education (% of GDP)' : c[c.Series=='Public expenditure on education (% of GDP)']['Value'],
              'Public expenditure on education (% of government expenditure)' : c[c.Series=='Public expenditure on education (% of government expenditure)']['Value']
             })

In [718]:
data2.dropna()

,Urban population (percent),Life expectancy at birth for both sexes (years),GDP per capita (US dollars),Public expenditure on education (% of GDP),Public expenditure on education (% of government expenditure)
Afghanistan,24.666667,60.9765,593.00000,3.54360,15.079333
Albania,56.633333,76.5630,4.11550,3.69630,12.457350
Angola,62.900000,55.2130,3.87850,3.47410,8.684800
Argentina,91.400000,75.2715,13.06325,5.29595,14.212250
Armenia,63.200000,73.3635,3.64825,3.00355,11.298850
...,...,...,...,...,...
Uruguay,94.900000,76.7125,14.97025,4.35530,14.926100
Uzbekistan,50.766667,69.6655,1.81100,6.41340,19.957200
Vanuatu,24.933333,69.1155,2.98500,4.69540,11.782700
Viet Nam,33.366667,74.7775,1.97200,5.39460,17.815000


In [719]:
data3 = pd.DataFrame({'Urban population (percent)' : a,
              'Life expectancy at birth for both sexes (years)': b,
              'GDP per capita (US dollars)' : d,                          
              'Public expenditure on education (% of GDP)' : c[c.Series=='Public expenditure on education (% of GDP)']['Value'],
             })

In [720]:
data3.dropna()

,Urban population (percent),Life expectancy at birth for both sexes (years),GDP per capita (US dollars),Public expenditure on education (% of GDP)
Afghanistan,24.666667,60.9765,593.00000,3.54360
Albania,56.633333,76.5630,4.11550,3.69630
Angola,62.900000,55.2130,3.87850,3.47410
Argentina,91.400000,75.2715,13.06325,5.29595
Armenia,63.200000,73.3635,3.64825,3.00355
...,...,...,...,...
Uruguay,94.900000,76.7125,14.97025,4.35530
Uzbekistan,50.766667,69.6655,1.81100,6.41340
Vanuatu,24.933333,69.1155,2.98500,4.69540
Viet Nam,33.366667,74.7775,1.97200,5.39460


In [727]:
data3.index.to_numpy()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia (Plurin. State of)', 'Bonaire, St. Eustatius & Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Cayman Islands', 'Central African Republic', 'Chad',
       'Channel Islands', 'Chile', 'China', 'China, Hong Kong SAR',
       'China, Macao SAR', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czechia',
       'Côte d’Ivoire', "Dem. People's Rep. Korea",
       'Dem. Rep. of the Congo', 'Denmark', 'Djibouti', 'Dominica',
    

Sada trebam paket pycountry kako bih dobio listu svih zemalja na svijetu. Paket se instalira naredbom:
conda install -c hargup/label/pypi pycountry

In [733]:
import pycountry